# PyCity Schools Analysis


---

In [2]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [3]:
# Calculate the total number of unique schools
school_count = school_data['School ID'].count()
school_count

15

In [4]:
# Calculate the total number of students
student_count = student_data['Student ID'].count()
student_count

39170

In [5]:
# Calculate the total budget
total_budget = school_data['budget'].sum()
total_budget

24649428

In [6]:
# Calculate the average (mean) math score
average_math_score = school_data_complete['math_score'].mean()
average_math_score

78.98537145774827

In [7]:
# Calculate the average (mean) reading score
average_reading_score = school_data_complete['reading_score'].mean()
average_reading_score

81.87784018381414

In [8]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [9]:
# Calculate the percentage of students who passeed reading (hint: look at how the math percentage was calculated)  
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
passing_reading_percentage

85.80546336482001

In [10]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

65.17232575950983

In [11]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame({
    "Total Schools": [school_count],
    "Total Students": [student_count],
    "Total Budget": [total_budget],
    "Average Math Score": [average_math_score],
    "Average Reading Score": [average_reading_score],
    "% Passing Math": [passing_math_percentage],
    "% Passing Reading": [passing_reading_percentage],
    "% Overall Passing": [overall_passing_rate]
})

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [12]:
# Use the code provided to select the school type
school_types = school_data.set_index(["school_name"])["type"]

In [13]:
# Calculate the total student count
per_school_counts = school_data_complete.groupby(['School ID'])['Student ID'].count()
per_school_counts.head()

School ID
0    2917
1    2949
2    1761
3    4635
4    1468
Name: Student ID, dtype: int64

In [14]:
# Calculate the total school budget and per capita spending
per_school_budget = school_data[['School ID', 'budget']]
per_school_capita = per_school_budget['budget'] / per_school_counts
per_school_capita.head()

0    655.0
1    639.0
2    600.0
3    652.0
4    625.0
dtype: float64

In [15]:
# Calculate the average test scores
per_school_math = school_data_complete.groupby(['School ID'])['math_score'].mean()
per_school_reading = school_data_complete.groupby(['School ID'])['reading_score'].mean()
per_school_math.head()

School ID
0    76.629414
1    76.711767
2    83.359455
3    77.289752
4    83.351499
Name: math_score, dtype: float64

In [18]:
# Calculate the number of schools with math scores of 70 or higher  
school_passing_math = per_school_math.loc[per_school_math >= 70]
school_passing_math

School ID
0     76.629414
1     76.711767
2     83.359455
3     77.289752
4     83.351499
5     83.274201
6     83.061895
7     77.048432
8     83.803279
9     83.839917
10    83.682222
11    76.842711
12    77.072464
13    77.102592
14    83.418349
Name: math_score, dtype: float64

In [20]:
# Calculate the number of schools with reading scores of 70 or higher
school_passing_reading = per_school_reading.loc[per_school_reading >= 70]

In [21]:
# Use the provided code to calculate the schools that passed both math and reading with scores of 70 or higher
passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]

In [24]:
# Use the provided code to calculate the passing rates
per_school_passing_math = school_passing_math / per_school_counts * 100
per_school_passing_reading = school_passing_reading / per_school_counts * 100
overall_passing_rate = passing_math_and_reading.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100

In [27]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = {
    'Total School Budget': total_budget,
    'Per School Budget': per_school_budget,
    'Per School Capita': per_school_capita,
    'Math Passing Rate': per_school_passing_math,
    'Reading Passing Rate': per_school_passing_reading,
    'Overall Passing Rate': overall_passing_rate
}

# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary

AttributeError: 'numpy.int64' object has no attribute 'map'

## Highest-Performing Schools (by % Overall Passing)

In [30]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = overall_passing_rate.sort_values(ascending= False)
top_schools.head(5)

Bailey High School     NaN
Cabrera High School    NaN
Figueroa High School   NaN
Ford High School       NaN
Griffin High School    NaN
dtype: float64

## Bottom Performing Schools (By % Overall Passing)

In [ ]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
top_schools = overall_passing_rate.sort_values(ascending= True)
bottom_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

In [32]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_graders_scores = ninth_graders.groupby(['school_name'])['math_score'].mean()
tenth_graders_scores = tenth_graders.groupby(['school_name'])['math_score'].mean()
eleventh_graders_scores = eleventh_graders.groupby(['school_name'])['math_score'].mean()
twelfth_graders_scores = twelfth_graders.groupby(['school_name'])['math_score'].mean()

# Use the code to select only the `math_score`.
ninth_grade_math_scores = ninth_graders_scores["math_score"]
tenth_grader_math_scores = tenth_graders_scores["math_score"]
eleventh_grader_math_scores = eleventh_graders_scores["math_score"]
twelfth_grader_math_scores = twelfth_graders_scores["math_score"]

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = {
    "Ninth Graders": ninth_grade_math_scores,
    "Tenth Graders": tenth_grader_math_scores,
    'Eleventh Graders': eleventh_grader_math_scores,
    'Twelfth Graders': twelfth_grader_math_scores
}

# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade

KeyError: 'math_score'

## Reading Score by Grade 

In [33]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_graders_scores = ninth_graders.groupby(['school_name'])['reading_score'].mean()
tenth_graders_scores = tenth_graders.groupby(['school_name'])['reading_score'].mean()
eleventh_graders_scores = eleventh_graders.groupby(['school_name'])['reading_score'].mean()
twelfth_graders_scores = twelfth_graders.groupby(['school_name'])['reading_score'].mean()

# Use the code to select only the `reading_score`.
ninth_grade_reading_scores = ninth_graders_scores["reading_score"]
tenth_grader_reading_scores = tenth_graders_scores["reading_score"]
eleventh_grader_reading_scores = eleventh_graders_scores["reading_score"]
twelfth_grader_reading_scores = twelfth_graders_scores["reading_score"]

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = {
    "Ninth Graders": ninth_grade_reading_scores,
    "Tenth Graders": tenth_grader_reading_scores,
    'Eleventh Graders': eleventh_grader_reading_scores,
    'Twelfth Graders': twelfth_grader_reading_scores
}

# Minor data wrangling
reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

KeyError: 'reading_score'

## Scores by School Spending

In [36]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [37]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

In [38]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df['Per Student Budget'], bins= spending_bins, labels= labels)

school_spending_df

KeyError: 'Per Student Budget'

In [34]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]

NameError: name 'school_spending_df' is not defined

In [ ]:
# Assemble into DataFrame
spending_summary = {
    'Average Math Score':s
    'Average Reading Score':
    '% Passing Math':
    '% Passing Reading'
    '% Overall Passing':
}

# Display results
spending_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


## Scores by School Size

In [40]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [39]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = pd.cut(bins= size_bins, labels= labels)
per_school_summary

NameError: name 'size_bins' is not defined

In [41]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"]).mean()["Average Math Score"]
size_reading_scores = per_school_summary.groupby(["School Size"]).mean()["Average Reading Score"]
size_passing_math = per_school_summary.groupby(["School Size"]).mean()["% Passing Math"]
size_passing_reading = per_school_summary.groupby(["School Size"]).mean()["% Passing Reading"]
size_overall_passing = per_school_summary.groupby(["School Size"]).mean()["% Overall Passing"]

AttributeError: 'dict' object has no attribute 'groupby'

In [ ]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = {
    "Average Math Score":size_math_scores,
    "Average Reading Score":size_reading_scores,
    "% Passing Math": size_passing_math,
    "% Passing Reading": size_passing_reading,
    "% Overall Passing": size_overall_passing
}

# Display results
size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

In [ ]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
type_math_scores = per_school_summary.groupby(['School Type'])['math_scores'].mean()
type_reading_scores = per_school_summary.groupby(['School Type'])['reading_scores'].mean()
type_passing_math = per_school_summary.groupby(['School Type'])['passing_math'].mean()
type_passing_reading = per_school_summary.groupby(['School Type'])['passing_reading'].mean()
type_overall_passing = per_school_summary.groupby(['School Type'])['overall_passing'].mean()

# Use the code provided to select new column data
average_math_score_by_type = type_math_scores["Average Math Score"]
average_reading_score_by_type = type_reading_scores["Average Reading Score"]
average_percent_passing_math_by_type = type_passing_math["% Passing Math"]
average_percent_passing_reading_by_type = type_passing_reading["% Passing Reading"]
average_percent_overall_passing_by_type = type_overall_passing["% Overall Passing"]

In [42]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = {
    'average_math_score_by_type': average_math_score_by_type,
    'average_reading_score_by_type': average_reading_score_by_type,
    'average_math_passing_by_type': average_percent_passing_math_by_type,
    'average_reading_passing_by_type': average_percent_passing_reading_by_type,
    'average_overall_passing_by_type': average_percent_overall_passing_by_type
}

# Display results
type_summary

NameError: name 'average_math_score_by_type' is not defined